# 1. Modulos
## 1.1 ¿Qué son?
Los módulos son archivos .py que contienen código Python (funciones, clases, variables) reutilizable.

## 1.2. ¿Para qué se usan?
Organizar código en piezas lógicas
Reutilizar funcionalidad sin reescribir
Mantener proyectos limpios y ordenados
Compartir código entre archivos
Tipos de módulos

In [ ]:
# Para trabajar con archivos de texto (.txt)
import io                      # manejo de flujos de texto y binarios, útil para encoding/decoding
import csv                     # lectura/escritura de archivos de texto delimitados (CSV/TSV)

# Para manejar rutas del sistema de forma segura y portátil
import os                      # operaciones del sistema (listado de directorios, variables de entorno)
from pathlib import Path       # manipulación orientada a objetos de rutas de archivos/directorios

# Para trabajar con fechas y tiempos
from datetime import datetime, date, timedelta  # clases básicas para fechas y operaciones con fechas
import time                    # utilidades relacionadas con tiempo (timestamps, sleep)

# Para leer/escribir Excel y manejo tabular
import pandas as pd            # lectura/escritura de Excel/CSV y manipulación de DataFrame
import openpyxl                # motor para leer/escribir archivos .xlsx (usado por pandas)

# Rutas y variables

In [ ]:
Periodo = "01_enero_2026"

r_sat_eps025 = r"C:\Users\karenmelo\Documents\Automatizaciones\data\SAT\EPS025"
r_sat_epsC25 = r"C:\Users\karenmelo\Documents\Automatizaciones\data\SAT\EPSC25"
r_salida = r"C:\Users\karenmelo\Documents\Automatizaciones\results"

# Carga Dataframes

In [ ]:
# cargar todos los .txt de r_sat_eps025 (ANSI, separador '|', sin encabezado, omitir primera línea, todas las columnas como str)
files = list(Path(r_sat_eps025).glob("*.txt"))
dfs = []
archivos_vacios = []

for f in files:
    try:
        df = pd.read_csv(f, sep='|', encoding='ANSI', header=None, skiprows=1, dtype=str)
        if len(df) > 0:  # Validar que no esté vacío
            df.insert(0, 'archivo', f.name)  # Agregar nombre del archivo en primera columna
            dfs.append(df)
        else:
            archivos_vacios.append(f.name)
    except pd.errors.EmptyDataError:
        archivos_vacios.append(f.name)

df_eps025 = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

if archivos_vacios:
    print(f"⚠️ Archivos vacíos omitidos (EPS025): {len(archivos_vacios)}")
    for archivo in archivos_vacios:
        print(f"  - {archivo}")

# cargar todos los .txt de r_sat_epsC25 (ANSI, separador '|', sin encabezado, omitir primera línea, todas las columnas como str)
files = list(Path(r_sat_epsC25).glob("*.txt"))
dfs = []
archivos_vacios = []

for f in files:
    try:
        df = pd.read_csv(f, sep='|', encoding='ANSI', header=None, skiprows=1, dtype=str)
        if len(df) > 0:  # Validar que no esté vacío
            df.insert(0, 'archivo', f.name)  # Agregar nombre del archivo en primera columna
            dfs.append(df)
        else:
            archivos_vacios.append(f.name)
    except pd.errors.EmptyDataError:
        archivos_vacios.append(f.name)

df_epsC25 = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

if archivos_vacios:
    print(f"⚠️ Archivos vacíos omitidos (EPSC25): {len(archivos_vacios)}")
    for archivo in archivos_vacios:
        print(f"  - {archivo}")

In [ ]:
print(f"EPS025 — registros: {df_eps025.shape[0]}, columnas: {df_eps025.shape[1]}")
print(f"EPSC25  — registros: {df_epsC25.shape[0]}, columnas: {df_epsC25.shape[1]}")

# Limpieza de datos

In [ ]:
# Agregar columna 'regimen' y unir ambos dataframes
df_eps025 = df_eps025.copy()
df_epsC25 = df_epsC25.copy()

df_eps025['regimen'] = 'subsidiado'
df_epsC25['regimen'] = 'contributivo'

df_unificado = pd.concat([df_eps025, df_epsC25], ignore_index=True, sort=False)

print(f"Unificado — registros: {df_unificado.shape[0]}, columnas: {df_unificado.shape[1]}")

## Filtras EPS vacias

In [ ]:
# Filtrar df_unificado para dejar solo filas cuya columna 1 esté en [6,7,14,19]
# Se asume que la columna existe y puede contener enteros o strings; se trabaja con str para robustez.

# Mostrar estado antes del proceso
print("ANTES — forma:", df_unificado.shape)
vc_antes = df_unificado[1].astype(str).value_counts().sort_index()
print("\nCategorias y conteos (columna 1) — antes:")
print(vc_antes)

# Valores a conservar (como strings)
valores_permitidos = ['6', '7', '14', '19']

# Aplicar filtro y sobrescribir df_unificado con el resultado filtrado
df_unificado = df_unificado[df_unificado[1].astype(str).isin(valores_permitidos)].copy()

# Mostrar estado después del proceso
print("\nDESPUÉS — forma:", df_unificado.shape)
vc_despues = df_unificado[1].astype(str).value_counts().reindex(valores_permitidos, fill_value=0)
print("\nCategorias y conteos (columna 1) — después (solo valores solicitados):")
print(vc_despues)

## Df_Traslados Salida

### EPS025

In [ ]:
# filtrar: columna 14 == "EPS025" y columna 16 no es vacio/0/null/"EPS025"/"EPSC25"
mask14 = df_unificado[14].astype(str).eq("EPS025")

s16 = df_unificado[16]
mask16 = (~s16.isna()) & (~s16.astype(str).str.strip().eq("")) & (~s16.astype(str).str.strip().eq("0")) & (~s16.astype(str).str.strip().isin(["EPS025", "EPSC25"]))

df_tras_salida_EPS025 = df_unificado[mask14 & mask16].copy()

In [ ]:
# eliminar registros donde columna 17 == "Activo Afiliación Oficio IPS" o "Activo Afiliación Oficio" e imprimir conteos
antes = len(df_tras_salida_EPS025)
print(f"ANTES — registros: {antes}")

mask = ~df_tras_salida_EPS025[17].astype(str).str.strip().isin([
    "Activo Afiliación Oficio IPS",
    "Activo afiliación de oficio"
])
df_tras_salida_EPS025 = df_tras_salida_EPS025[mask].copy()

despues = len(df_tras_salida_EPS025)
print(f"DESPUÉS — registros: {despues}")

In [ ]:
# eliminar registros donde columna 1 == "7" y la fecha de proceso (col 3) coincide con fecha efectiva (col 62)
antes = len(df_tras_salida_EPS025)
print(f"ANTES — registros: {antes}")

mask_col1_7 = df_tras_salida_EPS025[1].astype(str).eq('7')
mask_nonnull = df_tras_salida_EPS025[3].notna() & df_tras_salida_EPS025[62].notna()

# extraer fecha (YYYY-MM-DD) de la columna 3 (formato "YYYY-MM-DDTHH:MM:SS")
date_proc = df_tras_salida_EPS025.loc[mask_nonnull, 3].astype(str).str.split('T', n=1).str[0]
date_eff = df_tras_salida_EPS025.loc[mask_nonnull, 62].astype(str).str.strip()

mask_same_date = pd.Series(False, index=df_tras_salida_EPS025.index)
mask_same_date.loc[mask_nonnull] = date_proc.eq(date_eff)

to_remove = mask_col1_7 & mask_same_date
df_tras_salida_EPS025 = df_tras_salida_EPS025[~to_remove].copy()

despues = len(df_tras_salida_EPS025)
print(f"DESPUÉS — registros: {despues}")

In [ ]:
# Conteos por categoría y totales
def print_counts(df, name):
    s = df[1].astype(str)
    vc = s.value_counts().sort_index()
    print(f"\n{name} — total registros: {len(df)}")
    print(vc)
    print(f"Total desde value_counts: {vc.sum()}")

print_counts(df_tras_salida_EPS025, "df_tras_salida_EPS025")

### EPSC25

In [ ]:
# filtrar: columna 14 == "EPS025" y columna 16 no es vacio/0/null/"EPS025"/"EPSC25"
mask14 = df_unificado[14].astype(str).eq("EPSC25")

s16 = df_unificado[16]
mask16 = (~s16.isna()) & (~s16.astype(str).str.strip().eq("")) & (~s16.astype(str).str.strip().eq("0")) & (~s16.astype(str).str.strip().isin(["EPS025", "EPSC25"]))

df_tras_salida_EPSC25 = df_unificado[mask14 & mask16].copy()

In [ ]:
# eliminar registros donde columna 17 == "Activo Afiliación Oficio IPS" o "Activo Afiliación Oficio" e imprimir conteos
antes = len(df_tras_salida_EPSC25)
print(f"ANTES — registros: {antes}")

mask = ~df_tras_salida_EPSC25[17].astype(str).str.strip().isin([
    "Activo Afiliación Oficio IPS",
    "Activo afiliación de oficio"
])
df_tras_salida_EPSC25 = df_tras_salida_EPSC25[mask].copy()

despues = len(df_tras_salida_EPSC25)
print(f"DESPUÉS — registros: {despues}")

In [ ]:
# Conteos por categoría y totales
def print_counts(df, name):
    s = df[1].astype(str)
    vc = s.value_counts().sort_index()
    print(f"\n{name} — total registros: {len(df)}")
    print(vc)
    print(f"Total desde value_counts: {vc.sum()}")

print_counts(df_tras_salida_EPSC25, "df_tras_salida_EPSC25")

## Ingresos Nuevos

### EPS025

In [ ]:
# Filtrar df_unificado para dejar solo filas cuya columna 1 esté en [6, 7, 14, 19]
valores_permitidos = ['6', '7', '14', '19']
df_ingresos_SAT_eps025 = df_unificado[df_unificado[1].astype(str).isin(valores_permitidos)].copy()

print(f"df_ingresos_SAT_eps025 — registros: {len(df_ingresos_SAT_eps025)}")

In [ ]:
# Conteos por categoría y totales
def print_counts(df, name):
    s = df[1].astype(str)
    vc = s.value_counts().sort_index()
    print(f"\n{name} — total registros: {len(df)}")
    print(vc)
    print(f"Total desde value_counts: {vc.sum()}")

print_counts(df_tras_salida_EPS025, "df_tras_salida_EPS025")

In [ ]:
df_unificado

# Guardar infromación

In [ ]:
output_file = Path(r_salida) / f"Traslado Salida SAT {Periodo}.xlsx"
Path(r_salida).mkdir(parents=True, exist_ok=True)

# resumen por categoría de la columna 16 - EPS025
df_summary_eps025 = (
    df_tras_salida_EPS025[16]
    .astype(str)
    .value_counts()
    .sort_index()
    .rename_axis('categoria')
    .reset_index(name='cantidad')
)
df_summary_eps025 = pd.concat(
    [df_summary_eps025, pd.DataFrame([{'categoria': 'Total', 'cantidad': len(df_tras_salida_EPS025)}])],
    ignore_index=True
)

# resumen por categoría de la columna 16 - EPSC25
df_summary_epsc25 = (
    df_tras_salida_EPSC25[16]
    .astype(str)
    .value_counts()
    .sort_index()
    .rename_axis('categoria')
    .reset_index(name='cantidad')
)
df_summary_epsc25 = pd.concat(
    [df_summary_epsc25, pd.DataFrame([{'categoria': 'Total', 'cantidad': len(df_tras_salida_EPSC25)}])],
    ignore_index=True
)

from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils import get_column_letter

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # hojas detalle
    df_tras_salida_EPS025.to_excel(writer, sheet_name='Detalle EPS025', index=False)
    df_tras_salida_EPSC25.to_excel(writer, sheet_name='Detalle EPSC25', index=False)

    # hoja resumen: escribir las dos tablas una al lado de la otra
    startrow = 1        # dejar fila 1 para títulos
    col_eps025 = 0      # columna A
    col_epsc25 = 3      # columna D (separadas)

    df_summary_eps025.to_excel(writer, sheet_name='Resumen', startrow=startrow, startcol=col_eps025, index=False)
    df_summary_epsc25.to_excel(writer, sheet_name='Resumen', startrow=startrow, startcol=col_epsc25, index=False)

    wb = writer.book
    ws = writer.sheets['Resumen']

    # títulos encima de cada tabla
    ws.cell(row=1, column=col_eps025+1).value = "Traslado salida SAT EPS025"
    ws.cell(row=1, column=col_epsc25+1).value = "Traslado salida SAT EPSC25"

    # crear tablas con estilo para cada resumen
    n1 = len(df_summary_eps025)
    n2 = len(df_summary_epsc25)
    start_excel_row = startrow + 1  # pandas startrow es 0-based; Excel rows son 1-based

    # EPS025 table ref
    col1_letter = get_column_letter(col_eps025+1)
    col1_end_letter = get_column_letter(col_eps025 + df_summary_eps025.shape[1])
    table_end_row1 = start_excel_row + n1  # incluye header + filas de datos
    table_ref1 = f"{col1_letter}{start_excel_row}:{col1_end_letter}{table_end_row1}"
    table1 = Table(displayName="ResumenEPS025", ref=table_ref1)
    style1 = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                            showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    table1.tableStyleInfo = style1
    ws.add_table(table1)

    # EPSC25 table ref
    col2_letter = get_column_letter(col_epsc25+1)
    col2_end_letter = get_column_letter(col_epsc25 + df_summary_epsc25.shape[1])
    table_end_row2 = start_excel_row + n2
    table_ref2 = f"{col2_letter}{start_excel_row}:{col2_end_letter}{table_end_row2}"
    table2 = Table(displayName="ResumenEPSC25", ref=table_ref2)
    style2 = TableStyleInfo(name="TableStyleMedium4", showFirstColumn=False,
                            showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    table2.tableStyleInfo = style2
    ws.add_table(table2)

print(f"Guardado: {output_file}")